# Assignment

In [1]:
# importing necessary libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. downloading the data and making a clean dataframe

In [2]:
from bs4 import BeautifulSoup

In [3]:
import requests

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

from urllib.request import urlopen
html = urlopen(url)
soup = BeautifulSoup(html, "html.parser")

In [4]:
tbody = soup.find('tbody')

In [5]:
tbody.find('td').text

'\nM1ANot assigned\n\n'

In [6]:
import re
tbody.find('td').text.find('Not')

4

In [7]:
tbody.find_all('td')[0]

<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>

In [8]:
tbody.find_all('td')[0].text

'\nM1ANot assigned\n\n'

In [9]:
tbody.find_all('td')[3].text

'\nM4ANorth York(Victoria Village)\n\n'

In [10]:
toronto_data = []

In [11]:
for i in range(0, len(tbody.find_all('td'))):
    if tbody.find_all('td')[i].text.find('Not') > 0:
        continue
    else:
        toronto_data.append(tbody.find_all('td')[i].text)
        
toronto_data

['\nM3ANorth York(Parkwoods)\n\n',
 '\nM4ANorth York(Victoria Village)\n\n',
 '\nM5ADowntown Toronto(Regent Park / Harbourfront)\n\n',
 '\nM6ANorth York(Lawrence Manor / Lawrence Heights)\n\n',
 "\nM7AQueen's Park(Ontario Provincial Government)\n\n",
 '\nM9AEtobicoke(Islington Avenue)\n\n',
 '\nM1BScarborough(Malvern / Rouge)\n\n',
 '\nM3BNorth York(Don Mills)North\n\n',
 '\nM4BEast York(Parkview Hill / Woodbine Gardens)\n\n',
 '\nM5BDowntown Toronto(Garden District, Ryerson)\n\n',
 '\nM6BNorth York(Glencairn)\n\n',
 '\nM9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)\n\n',
 '\nM1CScarborough(Rouge Hill / Port Union / Highland Creek)\n\n',
 '\nM3CNorth York(Don Mills)South(Flemingdon Park)\n\n',
 '\nM4CEast York(Woodbine Heights)\n\n',
 '\nM5CDowntown Toronto(St. James Town)\n\n',
 '\nM6CYork(Humewood-Cedarvale)\n\n',
 '\nM9CEtobicoke(Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood)\n\n',
 '\nM1EScarborough(Guildwood / Morningsid

In [12]:
toronto_data[1][4]

'N'

In [13]:
toronto_data[1].split('(')[1]

'Victoria Village)\n\n'

In [14]:
postalcode = []
borough = []
neighborhood = []

for i in toronto_data:
    
    postalcode.append(i[1:4])
    borough.append(i.split('(')[0][4:])
    neighborhood.append(i.split('(')[1])
    
dict = {'PostalCode' : postalcode, 'Borough' : borough, 'Neighborhood' : neighborhood}
dict.keys()

dict_keys(['PostalCode', 'Borough', 'Neighborhood'])

In [15]:
df = pd.DataFrame(dict)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods)\n\n
1,M4A,North York,Victoria Village)\n\n
2,M5A,Downtown Toronto,Regent Park / Harbourfront)\n\n
3,M6A,North York,Lawrence Manor / Lawrence Heights)\n\n
4,M7A,Queen's Park,Ontario Provincial Government)\n\n


In [16]:
for i in range(0, len(df['Neighborhood'])):
    df['Neighborhood'][i] = df['Neighborhood'][i].split(')')[0]

In [17]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Queen's Park,Ontario Provincial Government


In [18]:
for i in range(0, len(df['Neighborhood'])):
    df['Neighborhood'][i] = df['Neighborhood'][i].replace(' / ', ',')
    
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [19]:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern,Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill,Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [20]:
df.shape

(103, 3)

## 2. Getting latitude and longitude

In [21]:
geo = pd.read_csv("Geospatial_Coordinates.csv")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
merge_df = pd.merge(df, geo, left_on = 'PostalCode', right_on = 'Postal Code')[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]

In [23]:
merge_df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill,Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
